In [268]:
from collections import OrderedDict
import csv
import datetime,time
import operator

class DataFrame(object):

    @classmethod
    def from_csv(cls, file_path, delimiting_character=',', quote_character='"'):
        with open(file_path, 'rU') as infile:
            reader = csv.reader(infile, delimiter=delimiting_character, quotechar=quote_character)
            data = []

            for row in reader:
                data.append(row)

            return cls(list_of_lists=data)

    def __init__(self, list_of_lists, header=True):
        if header:
            self.header = list_of_lists[0]
            self.data = list_of_lists[1:]
        else:
            self.header = ['column' + str(index + 1) for index, column in enumerate(list_of_lists[0])]
            self.data = list_of_lists



        ############# HW2 task 1 #############
        if len(self.header) != len(set(self.header)):
            raise Exception('There are duplicates!!!')
        ############# end task 1 #############

        ############# HW2 task 2 #############
        self.data=[[s.strip() for s in row] for row in self.data]
        ############# end task 2 #############
        
        def data_clean(self):
            
            for i in range(len(self)):
                for j in range(len(self[0])):
                    try:
                        self[i][j] = float(self[i][j].replace(',',''))
                    except ValueError:
                        try:
                            self[i][j] = datetime.datetime.strptime(self[i][j], '%m/%d/%y %H:%M')
                        except:
                            pass
            return self
        
        self.data = [OrderedDict(zip(self.header, row)) for row in data_clean(self.data)]   

                       
    def __getitem__(self, item):
        # this is for rows only
        if isinstance(item, (int, slice)):
            return self.data[item]

        # this is for columns only
        elif isinstance(item, str):
            return [row[item] for row in self.data]

        # this is for rows and columns
        elif isinstance(item, tuple):
            if isinstance(item[0], list) or isinstance(item[1], list):

                if isinstance(item[0], list):
                    rowz = [row for index, row in enumerate(self.data) if index in item[0]]
                else:
                    rowz = self.data[item[0]]

                if isinstance(item[1], list):
                    if all([isinstance(thing, int) for thing in item[1]]):
                        return [[column_value for index, column_value in enumerate([value for value in row.itervalues()]) if index in item[1]] for row in rowz]
                    elif all([isinstance(thing,str) for thing in item[1]]):
                        return [[row[column_name] for column_name in item[1]] for row in rowz]
                    else:
                        raise TypeError('What the hell is this?')

                else:
                    return [[value for value in row.itervalues()][item[1]] for row in rowz]
            else:
                if isinstance(item[1], (int, slice)):
                    return [[value for value in row.itervalues()][item[1]] for row in self.data[item[0]]]
                elif isinstance(item[1],str):
                    return [row[item[1]] for row in self.data[item[0]]]
                else:
                    raise TypeError('I don\'t know how to handle this...')


        elif isinstance(item, list):
            if isinstance(item[0],str):
                # for lists of column names
                if lambda x:x[item] in self.header:
                    return [[row[column_name] for column_name in item] for row in self.data]
                else:
                    print 'col_name is not exist.'
            elif isinstance(item[0],bool):
                return [row for boolen,row in zip(item,self.data) if boolen== True]
            else:
                print 'Error'
                
            
    def get_rows_where_column_has_value(self, column_name, value, index_only=False):
        if index_only:
            return [index for index, row_value in enumerate(self[column_name]) if row_value==value]
        else:
            return [row for row in self.data if row[column_name]==value]

    ############# HW2 task 3 #############
    def col_type(self,col_name):
        try:
            nums = [float(row[col_name]) for row in self.data]
            col_type = 'Number'
            return nums,col_type
        except ValueError:
            try:
                nums=[datetime.datetime.strptime(row[col_name], '%m/%d/%y %H:%M') for row in self.data]
                nums=[time.mktime(num.timetuple()) for num in nums]
                col_type = 'Date'
                return nums,col_type
            except:
                nums=''
                col_type ='Str'
                return nums,col_type
   
    
    def min(self,col_name):
        nums,col_type = self.col_type(col_name)
        if col_type == 'Number':
            print min(nums)
        elif col_type =='Date':
            print time.strftime('%m-%d-%y %H:%M',time.localtime(min(nums)))
        else:
            print ('Cannot be calculated')

    def max(self,col_name):
        nums,col_type = self.col_type(col_name)
        if col_type == 'Number':
            print max(nums)
        elif col_type =='Date':
            print time.strftime('%m-%d-%y %H:%M',time.localtime(max(nums)))
        else:
            print ('Cannot be calculated')

    def median(self,col_name):
        nums,col_type = self.col_type(col_name)
        nums = sorted(nums)
        center = int(len(nums) / 2)
        
        if len(nums) % 2 == 0:
            even = 1
        else:
            even = 0
        
        if col_type == 'Number':
            if even==0:
                print sum(nums[center - 1:center + 1]) / 2.0
            else:
                print nums[center]
        elif col_type =='Date':
            if even==0:
                print time.strftime('%m-%d-%y %H:%M',time.localtime(sum(nums[center - 1:center + 1]) / 2.0))
            else:
                print time.strftime('%m-%d-%y %H:%M',time.localtime(nums[center]))
        else:
            print ('Cannot be calculated')

    def mean(self,col_name):
        nums,col_type = self.col_type(col_name)
        if col_type == 'Number':
            print sum(nums) / len(nums)
        elif col_type =='Date':
            print time.strftime('%m-%d-%y %H:%M',time.localtime(sum(nums) / len(nums)))                
        else:
            print ('Cannot be calculated')

    def sum(self,col_name):
        nums,col_type = self.col_type(col_name)
        if col_type == 'Number':
            print sum(nums)
        elif col_type =='Date':
            print ('Date cannot sum.')             
        else:
            print ('Cannot be calculated')


    def std(self,col_name):
        nums,col_type = self.col_type(col_name)
        mean = sum(nums)/len(nums)
        if col_type == 'Number':
            print  (sum((x-mean)**2/len(nums) for x in nums))**0.5
        elif col_type =='Date':
            print ('Date do not need std.')             
        else:
            print ('Cannot be calculated')
    ############# end task 3 #############

    ############# HW2 task 4 #############
    
    def add_rows(self, list_of_lists):
        for new_list in list_of_lists:
            if len(new_list) == len(self.header):
                new_row = [OrderedDict(zip(self.header, row)) for row in list_of_lists]
                self.data = self.data + new_row
                return self
            else:
                print ('Wrong number of len')
    ############# end task 4 #############

    ############# HW2 task 5 #############
    
    def add_column(self, list_of_lists,col_name):
        # type: (object, object) -> object
        if len(list_of_lists) == len(self.data):
            self.header=self.header+col_name
            new_col_dict = [OrderedDict(zip(col_name,row)) for row in list_of_lists]
            for l in range(len(self.data)):
                for rowz in self.data:
                    rowz = self.data[l].update(new_col_dict[l]) 
            return self
        else:
            print ('Wrong number of len')
    ############# end task 5 #############

    ############# HW3 task 1 #############
    def sort_by(self, col_name, reverse):
        if isinstance(col_name,str):
            self.data = sorted(self.data,key= operator.itemgetter(col_name),reverse=reverse)
            return self.data
        elif isinstance(col_name,list):
            for i in range(len(col_name))[::-1]:
                self.data = sorted(self.data,key=operator.itemgetter(col_name[i]),reverse=reverse[i])
            return self.data
        else:
            print 'TypeError'

    ############# end task 1 #############

    ############# HW3 task 2 #############
    
class Series(object):
    
    def __eq__(self, other):
        ret_list = []
        for item in self:
            ret_list.append(item == other)
        return ret_list
       
    def __ne__(self, other):
        ret_list = []
        for item in self:
            ret_list.append(item != other)
        return ret_list
    
    def __lt__(self, other):
        ret_list = []
        for item in self:
            ret_list.append(item < other)
        return ret_list
    
    def __gt__(self, other):
        ret_list = []
        for item in self:
            ret_list.append(item > other)
        return ret_list  
    
    def __le__(self, other):
        ret_list = []
        for item in self:
            ret_list.append(item <= other)
        return ret_list   
    
    def __ge__(self, other):
        ret_list = []
        for item in self:
            ret_list.append(item >= other)
        return ret_list
    
    ############# end task 2 #############    
    
    ############# HW3 task 3 #############
    
#     def group_by(self,group,agg,agg_func):
        
        

In [249]:
df = DataFrame.from_csv('test_hw.csv')

In [269]:
df = DataFrame.from_csv('SalesJan2009.csv')

In [265]:
df['Payment_Type']

['Mastercard',
 'Visa',
 'Mastercard',
 'Visa',
 'Visa',
 'Visa',
 'Mastercard',
 'Mastercard',
 'Mastercard',
 'Visa',
 'Diners',
 'Amex',
 'Mastercard',
 'Visa',
 'Visa',
 'Diners',
 'Visa',
 'Diners',
 'Visa',
 'Diners',
 'Visa',
 'Diners',
 'Mastercard',
 'Visa',
 'Mastercard',
 'Visa',
 'Diners',
 'Visa',
 'Visa',
 'Visa',
 'Mastercard',
 'Mastercard',
 'Mastercard',
 'Diners',
 'Mastercard',
 'Amex',
 'Visa',
 'Visa',
 'Mastercard',
 'Amex',
 'Amex',
 'Mastercard',
 'Visa',
 'Visa',
 'Amex',
 'Visa',
 'Mastercard',
 'Mastercard',
 'Amex',
 'Diners',
 'Visa',
 'Mastercard',
 'Visa',
 'Visa',
 'Mastercard',
 'Visa',
 'Visa',
 'Visa',
 'Mastercard',
 'Diners',
 'Mastercard',
 'Visa',
 'Mastercard',
 'Mastercard',
 'Mastercard',
 'Mastercard',
 'Visa',
 'Visa',
 'Visa',
 'Diners',
 'Mastercard',
 'Visa',
 'Visa',
 'Visa',
 'Visa',
 'Visa',
 'Mastercard',
 'Visa',
 'Visa',
 'Visa',
 'Visa',
 'Diners',
 'Visa',
 'Mastercard',
 'Mastercard',
 'Mastercard',
 'Mastercard',
 'Visa',
 'Visa

In [261]:
ret_list = []
for item in df['Payment_Type']:
    ret_list.append(item == 'Visa')

In [262]:
ret_list

[False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 Tru

In [270]:
df[df['Price']==13000]

OrderedDict([('Transaction_date', datetime.datetime(2009, 1, 2, 6, 17)),
             ('Product', 'Product1'),
             ('Price', 1200.0),
             ('Payment_Type', 'Mastercard'),
             ('Name', 'carolina'),
             ('City', 'Basildon'),
             ('State', 'England'),
             ('Country', 'United Kingdom'),
             ('Account_Created', datetime.datetime(2009, 1, 2, 6, 0)),
             ('Last_Login', datetime.datetime(2009, 1, 2, 6, 8)),
             ('Latitude', 51.5),
             ('Longitude', -1.1166667)])

SyntaxError: invalid syntax (<ipython-input-69-b76f71cb04ab>, line 4)

SyntaxError: unexpected EOF while parsing (<ipython-input-88-7b2228fd2ba8>, line 1)

In [218]:
if lambda x:x[1,2] in [1,2,3]:
    print 'true'

true


250.0


In [267]:
dir(list)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__delslice__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setslice__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']